# Scripps Automated Shore Station (SASS) Self-Calibrating SeapHOx (SCS) live data viewer
### This script scrubs .dat files from the SCCOOS dr: https://sccoos.org/dr/data/. The SCS measures pH, oxygen, salinity and temperature. The pH values plotted here are calibrated using an average of the coefficients established pre-deployment and from the first automated tris calibration on 18 Feb 2022. Before using and sharing these values, please get permission from the author. 
### Created by: Taylor Wirth twirth@ucsd.edu
### Last edit: 2 March 2022

In [1]:
# load the .py scripts
%load_ext autoreload
%autoreload 1
%aimport get_all_dr
%aimport pHext_from_Vext_k0ext
%aimport pHint_from_Vint_k0int
%aimport plot_all

The last hour of data and plot of all the data will show below. Please wait!

In [2]:
# Get all the files for the Scripps Pier SCS. get_all_dr.py looks and grabs all data files for the Scripps Pier SCS, then converts it into a pandas DataFrame. 
# Printed below is the last hour of data. This may take ~20 seconds!
path = 'https://sccoos.org/dr/data/scripps_pier_scs/'
df = get_all_dr.get_all_dr(path)
df.tail(6)

,internet_datetime,IP,samp_type,samp_num,calib_num,calib_rep,date,time,vbatt,vtherm,...,e,f,g,SBEtemp,SBEcond,SBEsal,SBEday,SBEmon,SBEyear,SBEtime
1742,2022-03-02T20:00:26Z,172.16.117.226,0,1737,0,0,2022/03/02,20:00:10,15.72,0.57227,...,578.4,688.1,374.8,14.6920,4.10085,33.5289,2,Mar,2022,12:14:14
1743,2022-03-02T20:10:26Z,172.16.117.226,0,1738,0,0,2022/03/02,20:10:10,15.72,0.57250,...,577.0,687.8,374.4,14.7091,4.10232,33.5275,2,Mar,2022,12:24:14
1744,2022-03-02T20:20:26Z,172.16.117.226,0,1739,0,0,2022/03/02,20:20:10,15.72,0.57170,...,577.3,687.6,375.1,14.6592,4.09685,33.5207,2,Mar,2022,12:34:14
1745,2022-03-02T20:30:26Z,172.16.117.226,0,1740,0,0,2022/03/02,20:30:10,15.72,0.57070,...,589.1,687.5,368.7,14.9746,4.12576,33.5122,2,Mar,2022,12:44:14
1746,2022-03-02T20:40:26Z,172.16.117.226,0,1741,0,0,2022/03/02,20:40:10,15.72,0.57070,...,588.2,687.1,370.9,14.8666,4.11892,33.5430,2,Mar,2022,12:54:14
1747,2022-03-02T20:50:26Z,172.16.117.226,0,1742,0,0,2022/03/02,20:50:10,15.72,0.56907,...,584.1,686.9,365.5,15.1380,4.13902,33.4921,2,Mar,2022,13:04:14


In [30]:
k0int = -0.3961671
k0ext = -1.3843647

# calibration coefficients from first automated tris calibration on Feb 18, 2022
k0int_tris = -0.3970633
k0ext_tris = -1.3848532

print('k0int from predeployment =', k0int)
print('k0ext from predeployment =', k0ext)
print('k0int from tris =', k0int_tris)
print('k0ext from tris =', k0ext_tris)
print('mean k0int =', (k0int+k0int_tris)/2)
print('mean k0ext =', (k0ext+k0ext_tris)/2)

datetime = list(df['date'] + ' ' + df['time']) # combine date and time to use for plots
Vint = df.vint.astype(float)
Vext = df.vext.astype(float)
T_C = df.SBEtemp.astype(float)
sal = df.SBEsal.astype(float)
pHint_SBE = pHint_from_Vint_k0int.pHint_from_Vint_k0int((k0int+k0int_tris)/2, Vint, T_C)
pHint_tris = pHint_from_Vint_k0int.pHint_from_Vint_k0int(k0int_tris,Vint, T_C)
pHext_SBE = pHext_from_Vext_k0ext.pHext_from_Vext_k0ext((k0ext+k0ext_tris)/2, Vext, T_C, sal)
pHext_tris = pHext_from_Vext_k0ext.pHext_from_Vext_k0ext(k0ext_tris, Vext, T_C, sal)

k0int from predeployment = -0.3961671
k0ext from predeployment = -1.3843647
k0int from tris = -0.3970633
k0ext from tris = -1.3848532
mean k0int = -0.3966152
mean k0ext = -1.38460895


In [27]:
plot_all.plot_all(datetime,pHint_SBE,pHext_SBE,df.O2con,df.O2sat,df.SBEtemp,df.O2temp,df.pHtemp,df.SBEsal,df.press)